In [8]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# 1. Load CIFAR-100 data
(X_train, y_train), (X_test, y_test) = cifar100.load_data(label_mode='fine')

# 2. One-hot encode the labels
enc = OneHotEncoder(sparse_output=False)
y_train = enc.fit_transform(y_train)
y_test = enc.transform(y_test)

# 3. Define preprocessing function (resize + normalize)
def preprocess(image, label):
    image = tf.image.resize(image, [128, 128])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# 4. Shuffle training data before splitting
train_indices = np.arange(len(X_train))
np.random.shuffle(train_indices)

X_train = X_train[train_indices]
y_train = y_train[train_indices]

# 5. Compute split sizes
val_size = int(0.1 * len(X_train))
X_val = X_train[:val_size]
y_val = y_train[:val_size]
X_train = X_train[val_size:]
y_train = y_train[val_size:]

# 6. Create tf.data.Dataset pipelines
batch_size = 64

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=1000)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)


In [9]:
import tensorflow as tf
import tensorflow.keras.layers as tfl

# from preprocess import get_cifar100_datasets

# train_ds, val_ds, test_ds = get_cifar100_datasets(image_size=(128, 128), batch_size=32)

## Building ResNet-50

In [10]:
def identity_block(X, f, filters):
    X_shortcut=X

    X=tfl.Conv2D(filters=filters[0],kernel_size=1,strides=(1,1), padding='valid')(X)
    X=tfl.BatchNormalization(axis=3)(X, training=True)
    X=tfl.Activation('relu')(X)

    X=tfl.Conv2D(filters=filters[1],kernel_size=f,strides=(1,1), padding='same')(X)
    X=tfl.BatchNormalization(axis=3)(X, training=True)
    X=tfl.Activation('relu')(X)

    X=tfl.Conv2D(filters=filters[2],kernel_size=1,strides=(1,1), padding='valid')(X)
    X=tfl.BatchNormalization(axis=3)(X, training=True)

    X=tfl.Add()([X_shortcut,X])
    X=tfl.Activation('relu')(X)

    return X


In [11]:
def convolutional_block(X, f, filters, s=2):
    X_shortcut=X

    X=tfl.Conv2D(filters=filters[0],kernel_size=1,strides=(s,s), padding='valid')(X)
    X=tfl.BatchNormalization(axis=3)(X, training=True)
    X=tfl.Activation('relu')(X)

    X=tfl.Conv2D(filters=filters[1],kernel_size=f,strides=(1,1), padding='same')(X)
    X=tfl.BatchNormalization(axis=3)(X, training=True)
    X=tfl.Activation('relu')(X)

    X=tfl.Conv2D(filters=filters[2],kernel_size=1,strides=(1,1), padding='valid')(X)
    X=tfl.BatchNormalization(axis=3)(X, training=True)

    X_shortcut=tfl.Conv2D(filters=filters[2],kernel_size=1,strides=(s,s), padding='valid')(X_shortcut)
    X_shortcut=tfl.BatchNormalization(axis=3)(X_shortcut, training=True)

    X=tfl.Add()([X_shortcut,X])
    X=tfl.Activation('relu')(X)

    return X

In [12]:
def arch(input_shape):

    input_img = tf.keras.Input(shape=input_shape)

    layer =tfl.ZeroPadding2D((3, 3))(input_img)

    layer=tfl.Conv2D(filters=64,kernel_size=7,strides=(2,2))(layer)
    layer=tfl.BatchNormalization(axis=3)(layer, training=True)
    layer=tfl.Activation('relu')(layer)
    layer=tfl.MaxPooling2D((3, 3), strides=(2, 2))(layer)

    layer=convolutional_block(layer,3,[64,64,256],1)
    layer=identity_block(layer,3,[64,64,256])
    layer=identity_block(layer,3,[64,64,256])

    layer=convolutional_block(layer,3,[128,128,512],2)
    layer=identity_block(layer,3,[128,128,512])
    layer=identity_block(layer,3,[128,128,512])
    layer=identity_block(layer,3,[128,128,512])

    layer=convolutional_block(layer,3, [256, 256, 1024],2)
    layer=identity_block(layer,3, [256, 256, 1024])
    layer=identity_block(layer,3, [256, 256, 1024])
    layer=identity_block(layer,3, [256, 256, 1024])
    layer=identity_block(layer,3, [256, 256, 1024])
    layer=identity_block(layer,3, [256, 256, 1024])

    layer=convolutional_block(layer,3, [512, 512, 2048],2)
    layer=identity_block(layer,3, [512, 512, 2048])
    layer=identity_block(layer,3, [512, 512, 2048])

    layer=tfl.AveragePooling2D(pool_size=(2, 2),padding='same')(layer)
    layer=tfl.Flatten()(layer)

    outputs=tfl.Dense(units= 100 , activation='softmax')(layer)
    model = tf.keras.Model(inputs=input_img, outputs=outputs)
    return model

## training and evaluating the model

In [13]:
conv_model = arch((128, 128, 3)) 
conv_model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
conv_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 134, 134, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 64, 64, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 64, 64, 64)   256         conv2d_53[0][0]                  
____________________________________________________________________________________________

In [15]:
# Train the model
history = conv_model.fit(
    train_dataset,
    epochs=4,
    validation_data=val_dataset
)

Epoch 1/4
704/704 [==============================] - 129s 167ms/step - loss: 4.4185 - accuracy: 0.0906 - val_loss: 3.5577 - val_accuracy: 0.1582
Epoch 2/4
704/704 [==============================] - 116s 165ms/step - loss: 3.3001 - accuracy: 0.2054 - val_loss: 3.1113 - val_accuracy: 0.2430
Epoch 3/4
704/704 [==============================] - 117s 166ms/step - loss: 2.8821 - accuracy: 0.2850 - val_loss: 2.7985 - val_accuracy: 0.3012
Epoch 4/4
704/704 [==============================] - 120s 170ms/step - loss: 2.4922 - accuracy: 0.3637 - val_loss: 2.5842 - val_accuracy: 0.3550


In [16]:
test_loss, test_acc = conv_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.4f}")

157/157 [==============================] - 10s 61ms/step - loss: 2.5999 - accuracy: 0.3511
Test accuracy: 0.3511
